In [1]:
import findspark
import os
#findspark.init() 
SPARK_HOME='/opt/cloudera/parcels/CDH/lib/spark'
# SPARK_HOME='/home/qiany/.conda/envs/py37'
# os.environ['SPARK_HOME'] = '/home/qiany/.conda/envs/py37'
findspark.init(SPARK_HOME)

In [2]:
import time
import math
import copy
import csv
import json
import os
import codecs
import subprocess
#from hdfs import InsecureClient
import numpy as np
#from pyspark import SparkContext
from pyspark import SQLContext
from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql.functions import coalesce,broadcast,collect_list, collect_set, udf, array_remove, log, lit, first, col, array, sort_array,split, explode, desc, asc, row_number,isnan, when, count
from pyspark.sql.types import *
import rtree
from pyspark.sql import Window
#import igraph
#from igraph import Graph
import geofeather
from pyspark.storagelevel import StorageLevel

In [3]:
tin_file = input("Here is a programe to compute the Forman gradient, please input the absolute or relative path to your TIN file:")
print("\n********************")

# get the directory to the TIN file
tin_directory = os.path.dirname(tin_file)
print("tin_directory: ", tin_directory)

directory_type = input("Is the data stored in hdfs(0) or SigSpatial(1):") or "1"
if directory_type == '0':
    directory = 'hdfs_data'
else:
    directory = 'SigSpatial_data'
    
# get the basename to the TIN file
tin_basename = os.path.basename(tin_file) # input_vertices_2.off
print("tin_basename: ", tin_basename)

# get the filename of the TIN file
tin_filename = os.path.splitext(tin_basename)[0] # input_vertices_2
print("tin_filename: ", tin_filename)

# get the type of TIN file: off, tri, etc
tin_extension = os.path.splitext(tin_basename)[1] # .off
print("tin_extension: ", tin_extension)
print("\n********************")

Here is a programe to extract V1-Paths, please input the absolute or relative path to your TIN file: SigSpatial_data/Canyon_Lake_Gorge_TX.off



********************
tin_directory:  SigSpatial_data
tin_basename:  Canyon_Lake_Gorge_TX.off
tin_filename:  Canyon_Lake_Gorge_TX
tin_extension:  .off

********************
This is a TIN file in ".off" format


In [4]:
# filtra = input("Do you have filtration data?")
# filtra = 'yes'

# if filtra.lower() == 'no':
#     Basic_Data = input("Do you have basic pts and tri data?")
    
# Num_executor = input("spark.executor.instances:")
# Num_core_per_executor = input("spark.executor.cores:")
# Memory_executor = input("spark.executor.memory? Please end with 'g':")
# MemoryOverhead_executor = input("spark.executor.memoryOverhead? Please end with 'g':")

# Num_core_per_driver = Num_core_per_executor
# Memory_driver = Memory_executor
# MemoryOverhead_driver = MemoryOverhead_executor
Num_core_per_driver = '5'
Memory_driver = '20g'
MemoryOverhead_driver = '4g'


Num_shuffle_partitions = input("spark.sql.shuffle.partitions:")
# StorageFraction = input("spark.memory.storageFraction:")
# Num_subgraphs = int(input("How many subgraphs:"))

Num_executor = '64'
Num_core_per_executor = '5'
Memory_executor = '64g'
MemoryOverhead_executor = '8g'
# Num_shuffle_partitions = '20'
# StorageFraction = '0.5'

spark.sql.shuffle.partitions: 200


In [5]:
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import geopandas as gpd
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, ArrayType, MapType
from shapely.geometry import Point
from shapely.geometry import Polygon

from sedona.register import SedonaRegistrator
from sedona.core.SpatialRDD import SpatialRDD, PointRDD, CircleRDD, PolygonRDD, LineStringRDD
from sedona.core.enums import FileDataSplitter
from sedona.utils.adapter import Adapter
from sedona.core.spatialOperator import KNNQuery
from sedona.core.spatialOperator import JoinQuery
from sedona.core.spatialOperator import JoinQueryRaw
from sedona.core.spatialOperator import RangeQuery
from sedona.core.spatialOperator import RangeQueryRaw
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import WkbReader
from sedona.core.formatMapper import WktReader
from sedona.core.formatMapper import GeoJsonReader
from sedona.sql.types import GeometryType
from sedona.core.enums import GridType
from sedona.core.SpatialRDD import RectangleRDD
from sedona.core.enums import IndexType
from sedona.core.geom.envelope import Envelope
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

In [6]:
os.environ['PYSPARK_PYTHON'] = "./environment/bin/python"
#os.environ['PYSPARK_PYTHON'] = "/home/qiany/.conda/envs/py37/bin/python"
os.environ['YARN_CONF_DIR'] = "/opt/cloudera/parcels/CDH/lib/spark/conf/yarn-conf"

In [7]:
'''
spark.executor.cores: # Number of concurrent tasks an executor can run, euqals to the number of cores to use on each executor
spark.executor.instances: # Number of executors for the spark application
spark.executor.memory: # Amount of memory to use for each executor that runs the task
spark.executor.memoryOverhead:
spark.driver.cores: # Number of cores to use for the driver process; the default number is 1
spark.driver.memory: # Amount of memory to use for the driver
spark.driver.maxResultSize: to define the maximum limit of the total size of the serialized result that a driver can store for each Spark collect action
spark.default.parallelism: # Default number of partitions in RDDs returned by transformations like join, reduceByKey, and parallelize when not set by user. It can be set as spark.executor.instances * spark.executor.cores * 2
spark.sql.shuffle.partitions: determine how many partitions are used when data is shuffled between nodes, e.g., joins or aggregations. usually 1~5 times of executor.instances * executor.cores
spark.memory.storageFraction: determines the fraction of the heap space that is allocated to caching RDDs and DataFrames in memory.
spark.kryoserializer.buffer.max: determine the maximum of data that can be serialized at once; this must be larger than any object we attempt to serialize
spark.rpc.message.maxSize: # Maximum message size (in MiB) to allow in "control plane" communication; generally only applies to map output size information sent between executors and the driver. To communicate between the nodes, Spark uses a protocol called RPC (Remote Procedure Call), which sends messages back and forth. The spark.rpc.message.maxSize parameter limits how big these messages can be. 
spark.sql.broadcastTimeout: Spark will wait for this amount of time before giving up on broadcasting a table. Broadcasting can take a long time if the table is large or if there is a shuffle operation before it.
spark.sql.autoBroadcastJoinThreshold: Spark will broadcast a table to all worker nodes when performing a join if its size is less than this value; -1 means disabling broadcasting
'''
# .config('spark.memory.storageFraction', StorageFraction) \

date = time.strftime("%m,%d,%Y")
date_name = date.split(',')[0] + date.split(',')[1] + date.split(',')[2]

hour = time.strftime("%H,%M")
hour_name = hour.split(',')[0] + hour.split(',')[1]

spark_app_name = "Save_PtsFiltra_DataFrame_" + tin_filename + '_' + date_name + '_' + hour_name
print("spark_app_name:", spark_app_name)

spark = SparkSession \
.builder \
.appName(spark_app_name) \
.master('yarn') \
.config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
.config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
.config('spark.jars','sedona-core-2.4_2.11-1.0.0-incubating.jar,sedona-sql-2.4_2.11-1.0.0-incubating.jar,sedona-python-adapter-2.4_2.11-1.0.0-incubating.jar,sedona-viz-2.4_2.11-1.0.0-incubating.jar,geotools-wrapper-geotools-24.0.jar,graphframes-0.8.0-spark2.4-s_2.11.jar') \
.config('spark.executor.cores', Num_core_per_executor) \
.config('spark.executor.instances', Num_executor) \
.config('spark.executor.memory', Memory_executor) \
.config('spark.executor.memoryOverhead', MemoryOverhead_executor) \
.config('spark.driver.cores', Num_core_per_driver) \
.config('spark.driver.memory', Memory_driver) \
.config('spark.driver.memoryOverhead', MemoryOverhead_driver) \
.config('spark.driver.maxResultSize', '0') \
.config('spark.dynamicAllocation.enabled', 'false') \
.config('spark.network.timeout', '10000001s') \
.config('spark.executor.heartbeatInterval', '10000000s') \
.config('spark.sql.shuffle.partitions', Num_shuffle_partitions) \
.config("spark.default.parallelism", '20') \
.config("spark.kryoserializer.buffer.max", "1024mb") \
.config('spark.rpc.message.maxSize', '256') \
.config("spark.sql.broadcastTimeout", "36000") \
.config("spark.sql.autoBroadcastJoinThreshold", "-1") \
.config('spark.yarn.dist.archives', '/local/data/yuehui/py37.tar.gz#environment') \
.getOrCreate()

spark_app_name: Save_PtsFiltra_DataFrame_Canyon_Lake_Gorge_TX_07112024_2147


### read the TIN and save its vertices and triangles as csv files

In [ ]:
t0 = time.time()
# url_in = '/home/qiany/yuehui/pyspark/data/B4.off'
# url_pt_origin_csv = '/home/qiany/yuehui/pyspark/data/B4_pts_origin.csv'
# url_tri_origin_csv = '/home/qiany/yuehui/pyspark/data/B4_tri_origin.csv'
# rank points with filtration.txt file

url_in = tin_file
url_pt_origin_csv = tin_directory + '/' + tin_filename + '_pts_origin.csv'
url_tri_origin_csv = tin_directory + '/' + tin_filename + '_filtra_tri_origin.csv'

if tin_extension == ".off":
    with open(url_in) as infile:
        trash = infile.readline()
        line = (infile.readline()).split()
        vertices_num = int(line[0])
        triangles_num = int(line[1])
        print("vnum: {}, tnum:{}".format(vertices_num, triangles_num))
        with open(url_pt_origin_csv, 'w', newline='') as ofs_pts:
            writer = csv.writer(ofs_pts)
            for l in range(vertices_num):
                line = infile.readline().split()
                v = [float(line[0]),float(line[1]),float(line[2]),l] # x,y,ele,self_index
                writer.writerow(v)
            ofs_pts.close()
            
        with open(url_tri_origin_csv, 'w', newline='') as ofs_tri:
            writer = csv.writer(ofs_tri)
            tri_order = 0
            for l in range(triangles_num):
                line = infile.readline().split()
                tri = [int(line[1]),int(line[2]),int(line[3]), tri_order]        
                writer.writerow(tri)
                tri_order += 1
            ofs_tri.close()    
        infile.close()

if tin_extension == ".tri":
    with open(url_in) as infile:
        # trash = infile.readline()
        line = (infile.readline()).split()
        vertices_num = int(line[0])
        # triangles_num = int(line[1])
        print("vnum: {}".format(vertices_num))
        with open(url_pt_origin_csv, 'w', newline='') as ofs_pts:
            writer = csv.writer(ofs_pts)
            for l in range(vertices_num):
                line = infile.readline().split()
                v = [float(line[0]),float(line[1]),float(line[2]),l] # x,y,ele,self_index, ele_order
                writer.writerow(v)
            ofs_pts.close()
            
        line = (infile.readline()).split()
        triangles_num = int(line[0])
        print("tnum: {}".format(triangles_num))
        with open(url_tri_origin_csv, 'w', newline='') as ofs_tri:
            writer = csv.writer(ofs_tri)
            tri_order = 0
            for l in range(triangles_num):
                line = infile.readline().split()
                tri = [int(line[0]),int(line[1]),int(line[2]), tri_order]        
                writer.writerow(tri)
                tri_order += 1
            ofs_tri.close()    
        infile.close()
        
t1 = time.time()
print("time cost:", t1-t0)

### sort the vertices of a mesh and get its filtration

In [15]:
# StructType is a collection of StructField’s which is used to define the column name, data type, and a flag for nullable or not.
# StructType: https://sparkbyexamples.com/pyspark/pyspark-sql-types-datatype-with-examples/
schema_ver_origin = StructType([ \
    StructField("x",FloatType(),True), \
    StructField("y",FloatType(),True), \
    StructField("ele",FloatType(),True), \
    StructField("self_index",LongType(),True) \
  ])

# hdfs_tin_pts_origin = "hdfs_data" + "/" + tin_filename + '_pts_origin.csv'
hdfs_tin_pts_origin = "SigSpatial_data" + "/" + tin_filename + '_pts_origin.csv'

df_ver_origin = spark.read.format("csv") \
      .option("header", False) \
      .schema(schema_ver_origin)\
      .load(hdfs_tin_pts_origin)

# print("Number of partitions:", df_ver_origin.rdd.getNumPartitions())
# df_ver_origin.printSchema()

# define a window for the ordering
# row_number() function along with partitionBy() of other column populates the row number by group
# Since we want to order the whole DataFrame, so we don't need the partitionBy() function
w = Window().orderBy(col('ele').asc())
df_ver_order = df_ver_origin.withColumn("self_order", F.row_number().over(w) -1) # let the row number start from 0
# df_ver_order = df_ver_origin.withColumn("self_order", col("self_order").cast(LongType())) # change the data type of self_order to LongType

df_ver_order.printSchema()

root
 |-- x: float (nullable = true)
 |-- y: float (nullable = true)
 |-- ele: float (nullable = true)
 |-- self_index: long (nullable = true)
 |-- self_order: integer (nullable = true)



### write the DataFrame to HDFS

In [22]:
# save df_ver_order
t0 = time.time()

file_df_ver_order_hdfs = 'Sigspatial_data' + '/' + tin_filename + '_filtra_pts_origin.csv'

# df_ver_order.write.mode('overwrite').options(header='False', delimiter=',').csv(file_df_ver_order_hdfs)
df_ver_order.write.csv(file_df_ver_order_hdfs)

t1 = time.time()
print("Time cost for saving df_ver_format to a csv file:", t1-t0)

Time cost for saving df_ver_format to a csv file: 0.8290736675262451


In [8]:
# read_tri_order() is a function used to read triangles from a csv file
def read_tri_order(filtra, directory, tin_filename):
    '''
    this function has two input parameters.
    filtra: 'yes' or 'no', yes means that the input csv file is ordered by default
    directory: a string denoting the directory to a TIN file
    tin_filename: a string denoting the file name of a tin without extension, e.g., 827_monviso
    '''
    if filtra.lower() == 'yes':
        hdfs_tin_tri_origin = directory + "/" + tin_filename + '_filtra_tri_origin.csv'
    else: # filtra.lower() == 'no'
        hdfs_tin_tri_origin = directory + "/" + tin_filename + '_tri_origin.csv'
        
    schema_tri_origin = StructType([ \
        StructField("v1",IntegerType(),True), \
        StructField("v2",IntegerType(),True), \
        StructField("v3",IntegerType(),True), \
        StructField("tri_order",IntegerType(),True) \
      ])

    df_tri_origin = spark.read.format("csv") \
          .option("header", False) \
          .schema(schema_tri_origin)\
          .load(hdfs_tin_tri_origin)
    print("Number of partitions for df_tri_origin:", df_tri_origin.rdd.getNumPartitions())
        
    return df_tri_origin

In [12]:
# read triangles
df_tri_origin = read_tri_order(filtra, directory, tin_filename)
df_tri_origin.printSchema()

Number of partitions for df_tri_origin: 27
root
 |-- v1: integer (nullable = true)
 |-- v2: integer (nullable = true)
 |-- v3: integer (nullable = true)
 |-- tri_order: integer (nullable = true)



In [13]:
# replace_ver() replaces the original index of each vertex with filtra value in df_tri_origin
def replace_ver(df_ver_order, df_tri_origin):
    '''
    df_ver_order: a DataFrame storing sorted vertices with filtra values
    df_tri_origin: a DataFrame storing triangles from a TIN
    '''
    df_tri_order_v1 = df_tri_origin.join(df_ver_order, df_tri_origin.v1 == df_ver_order.self_index, "inner")
    df_tri_order_v1 = df_tri_order_v1.select(col("tri_order"), col("v1"), col("v2"), col("v3"),col("self_order").alias("r1"), col("ele").alias("r1_ele"))

    df_tri_order_v2 = df_tri_order_v1.join(df_ver_order, df_tri_order_v1.v2 == df_ver_order.self_index, "inner")
    df_tri_order_v2 = df_tri_order_v2.select(col("tri_order"), col("v1"), col("v2"), col("v3"), col("r1"), col("self_order").alias("r2"), col("r1_ele"), col("ele").alias("r2_ele"))

    df_tri_order_v3 = df_tri_order_v2.join(df_ver_order, df_tri_order_v2.v3 == df_ver_order.self_index, "inner")
    df_tri_order_v3 = df_tri_order_v3.select(col("tri_order"), col("v1"), col("v2"), col("v3"), col("r1"), col("r2"), col("self_order").alias("r3"), col("r1_ele"), col("r2_ele"), col("ele").alias("r3_ele"))

    df_tri_order = df_tri_order_v3.select(col("tri_order"), col("r1"), col("r2"), col("r3"), col("r1_ele"), col("r2_ele"), col("r3_ele"))

    return df_tri_order

In [16]:
# replaces original vertex index with filtra values
df_tri_order = replace_ver(df_ver_order, df_tri_origin)
df_tri_order.printSchema()

root
 |-- tri_order: integer (nullable = true)
 |-- r1: integer (nullable = true)
 |-- r2: integer (nullable = true)
 |-- r3: integer (nullable = true)
 |-- r1_ele: float (nullable = true)
 |-- r2_ele: float (nullable = true)
 |-- r3_ele: float (nullable = true)



In [ ]:
# save df_tri_order
t0 = time.time()

file_df_tri_order_hdfs = 'Sigspatial_data' + '/' + tin_filename + '_filtra_tri_origin.csv'

# df_ver_order.write.mode('overwrite').options(header='False', delimiter=',').csv(file_df_ver_order_hdfs)
df_tri_order.write.csv(file_df_tri_order_hdfs)

t1 = time.time()
print("Time cost for saving df_ver_format to a csv file:", t1-t0)